### 15336129 马生科

In [2]:
import pandas as pd
import numpy as np

In [3]:
from scipy import stats

In [77]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('../data.txt')

In [10]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### 一.

#### 线性回归拟合

In [58]:
from sklearn.linear_model import LinearRegression
X = data[['age','bmi','children']].values
y = data['charges'].values
X_train, y_train = X[:-5], y[:-5]
X_test,y_test = X[-5:],y[-5:]

In [60]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### 预测

In [78]:
y_test_pred = lr.predict(X_test)
df = data.iloc[-5:,]
df['预测值']=y_test_pred

In [79]:
df

,age,sex,bmi,children,smoker,region,charges,预测值
1333,50,male,30.97,3,no,northwest,10600.5483,16989.312781
1334,18,female,31.92,0,no,northeast,2205.9808,8059.725787
1335,18,female,36.85,0,no,southeast,1629.8335,9705.113218
1336,21,female,25.80,0,no,southwest,2007.9450,6730.408091
1337,61,female,29.07,0,yes,northwest,29141.3603,17331.533438


In [81]:
import statsmodels.api as sm

In [90]:
mod = sm.OLS(y_train, X_train)
res = mod.fit()

In [101]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
prstd, iv_l, iv_u = wls_prediction_std(res,X_test,alpha=0.05)

In [109]:
df['lower']=iv_l
df['upper']=iv_u

In [110]:
df

,age,sex,bmi,children,smoker,region,charges,预测值,lower,upper
1333,50,male,30.97,3,no,northwest,10600.5483,16989.312781,-5959.954530,39007.358091
1334,18,female,31.92,0,no,northeast,2205.9808,8059.725787,-13538.729053,31424.971860
1335,18,female,36.85,0,no,southeast,1629.8335,9705.113218,-12737.954702,32256.152246
1336,21,female,25.80,0,no,southwest,2007.9450,6730.408091,-13924.872086,31005.225915
1337,61,female,29.07,0,yes,northwest,29141.3603,17331.533438,-5274.740633,39704.025393


### 二. 方差分析

### （1）性别对个人医疗费用是否显著

In [21]:
model = ols('charges ~ sex',data).fit()
charges_on_sex = anova_lm(model)

In [22]:
charges_on_sex

,df,sum_sq,mean_sq,F,PR(>F)
sex,1.0,6.435902e+08,6.435902e+08,4.399702,0.036133
Residual,1336.0,1.954306e+11,1.462804e+08,NaN,NaN


In [34]:
charges_on_sex  = charges_on_sex.rename(
    {'df':'自由度','sum_sq':'平方和','mean_sq':'均方','F':'F值','PR(>F)':'p值'},axis=1)

In [24]:
charges_on_sex

,自由度,平方和,均方,F值,p值
sex,1.0,6.435902e+08,6.435902e+08,4.399702,0.036133
Residual,1336.0,1.954306e+11,1.462804e+08,NaN,NaN


### （2） 双因素方差分析

In [30]:
model = ols('charges ~ sex + smoker',data).fit()
charges_on_sex_and_smoker = anova_lm(model)

In [31]:
charges_on_sex_and_smoker

,df,sum_sq,mean_sq,F,PR(>F)
sex,1.0,6.435902e+08,6.435902e+08,11.524608,7.069618e-04
smoker,1.0,1.208777e+11,1.208777e+11,2164.527244,1.190490e-281
Residual,1335.0,7.455290e+10,5.584487e+07,NaN,NaN


In [32]:
charges_on_sex_and_smoker  = charges_on_sex_and_smoker.rename(
    {'df':'自由度','sum_sq':'平方和','mean_sq':'均方','F':'F值','PR(>F)':'p值'},axis=1)

In [33]:
charges_on_sex_and_smoker

,自由度,平方和,均方,F值,p值
sex,1.0,6.435902e+08,6.435902e+08,11.524608,7.069618e-04
smoker,1.0,1.208777e+11,1.208777e+11,2164.527244,1.190490e-281
Residual,1335.0,7.455290e+10,5.584487e+07,NaN,NaN
